<a href="https://colab.research.google.com/github/jnpos/group8_ECG/blob/Feature-selection/ECG_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/jveenland/tm10007_ml.git

fatal: destination path 'tm10007_ml' already exists and is not an empty directory.


In [ ]:
# General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn import metrics
import zipfile
import os
import pandas as pd

# Metrics
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import r2_score

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import model_selection

In [ ]:
with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 827
The number of columns: 9001


In [ ]:
# Find column with label 
bool_cols = [col for col in data 
             if np.isin(data[col].dropna().unique(), [0, 1]).all()]
loc_label = data.columns.get_loc('label')

# Determine data and output 
y = data['label']
x = pd.DataFrame()
x = data.drop(data.columns[loc_label],axis=1)

# Split test and trainingsdata 
#sss = model_selection.StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=0)
#sss.get_n_splits(X, y)
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y,test_size=0.25,random_state=0,stratify=y)

print(len(x_train))

print(0.75*620)

620
465.0


In [ ]:
# Reduce dimensionality
n_samples = len(x_train)
n_features = len(x_train.columns)
n_features = min((n_samples-1), n_features)
print(n_features)

p = PCA(n_components=n_features)
p = p.fit(x_train)
x_pca = p.transform(x_train)

print(x_pca.shape)

#fig = plt.figure(figsize=(24,8))
#plt.scatter(x[:,0], x[:,1], marker='o', c=y_train, s=25, edgecolor = 'k')

619
(620, 619)


In [ ]:
# LASSO regression

from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(x_pca, y_train)
model_lsvc = SelectFromModel(lsvc, prefit=True)
X_lasso = model_lsvc.transform(x_pca)
X_lasso.shape

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


(620, 270)

In [ ]:
# Feature selection univariate & f_classif
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif,k="all")
selector.fit(x_pca,y_train)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()

dataframe = pd.DataFrame()
dataframe = selector.pvalues_

print(len(selector.pvalues_[selector.pvalues_<0.05]))

# define feature selection
fs = SelectKBest(score_func=f_classif, k=len(selector.pvalues_[selector.pvalues_<0.05]))
# apply feature selection
X_univariate = fs.fit_transform(x_pca, y_train)
print(X_univariate.shape)





28
(620, 28)


In [ ]:
# Elastic net

from sklearn.linear_model import ElasticNet

ENreg = ElasticNet(alpha=1, l1_ratio=0.5).fit(x_pca,y_train)
model_ENreg = SelectFromModel(ENreg, prefit=True)
x_ENreg = model_ENreg.transform(x_pca)

x_ENreg.shape

(620, 104)

In [ ]:
# Feature selection Greedy - runt na 30 min nog niet
from sklearn.feature_selection import SequentialFeatureSelector

knn = KNeighborsClassifier(n_neighbors=4)
selector = SequentialFeatureSelector(knn, n_features_to_select=10, direction ="forward", scoring='accuracy', cv=5)
selector.fit(x_train,y_train)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()

dataframe = pd.DataFrame()
dataframe = selector.pvalues_

#print(len(selector.pvalues_[selector.pvalues_<0.05]))


KeyboardInterrupt: ignored